<a href="https://colab.research.google.com/github/coll-j/B21-CAP0001/blob/ml/ML/Testing_Binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install PySastrawi

     |████████████████████████████████| 215kB 6.9MB/s 


In [2]:
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
import re

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


In [3]:
!wget https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/new_kamusalay.csv

--2021-05-30 07:41:52--  https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/new_kamusalay.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 285941 (279K) [text/plain]
Saving to: ‘new_kamusalay.csv’

new_kamusalay.csv   100%[===================>] 279.24K  --.-KB/s    in 0.02s   

2021-05-30 07:41:53 (11.0 MB/s) - ‘new_kamusalay.csv’ saved [285941/285941]



In [4]:
kamus_alay = pd.read_csv('new_kamusalay.csv', encoding="ISO-8859-1", header=None)
kamus_alay.head()

,0,1
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [5]:
kamus_alay_dict = {}
for i, row in kamus_alay.iterrows():
  kamus_alay_dict[row[0]] = row[1]

kamus_alay_dict

{'anakjakartaasikasik': 'anak jakarta asyik asyik',
 'pakcikdahtua': 'pak cik sudah tua',
 'pakcikmudalagi': 'pak cik muda lagi',
 't3tapjokowi': 'tetap jokowi',
 '3x': 'tiga kali',
 'aamiin': 'amin',
 'aamiinn': 'amin',
 'aamin': 'amin',
 'aammiin': 'amin',
 'abis': 'habis',
 'abisin': 'habiskan',
 'acau': 'kacau',
 'achok': 'ahok',
 'ad': 'ada',
 'adek': 'adik',
 'adl': 'adalah',
 'adlah': 'adalah',
 'adlh': 'adalah',
 'ado': 'ada',
 'aduhh': 'aduh',
 'aer': 'air',
 'afdol': 'afdal',
 'agamaataualqur': 'agama alquran',
 'agm': 'agama',
 'agma': 'agama',
 'ahaha': 'haha',
 'ahahaha': 'haha',
 'ahehehehe': 'hehe',
 'ahir': 'akhir',
 'ahirnya': 'akhirnya',
 'ahk': 'ahok',
 'ahlamdulillah': 'alhamdulillah',
 'ahli2': 'para ahli',
 'ahlusunnah': 'ahlus sunah',
 'ahmaddani': 'ahmad dhani',
 'aho': 'ahok',
 'ahoax': 'ahok',
 'ahoaxx': 'ahok',
 'ahog': 'ahok',
 'ahokataudjarot': 'ahok djarot',
 'ahokbebanijokowi': 'ahok beban jokowi',
 'ahokbtp': 'ahok basuki tjahaja purnama',
 'ahokditolakw

In [29]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [23]:
with open('tokenizer_stem.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [41]:
def preprocess(sents, max_len=58):
  lower_sents = [s.lower() for s in sents]
  cleaned = lower_sents
  cleaned = list(map(lambda s: stemmer.stem(s), cleaned)) # stemming
  query = r'[^a-zA-Z\d\s]' # ambil alphanumeric saja
  cleaned = list(map(lambda s: re.sub(query, ' ', s), cleaned))
  query = r'(?:^| )\w(?:$| )'
  cleaned = list(map(lambda s: re.sub(query, ' ', s), cleaned)) # hilangkan single letter
  query = r" +"
  cleaned = list(map(lambda s: re.sub(query, ' ', s.strip()), cleaned)) # hilangkan trailing spaces
  seq = tokenizer.texts_to_sequences(cleaned)
  seq = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_len)

  return seq

In [10]:
model_fn = 'binary_repo_83_41.h5'
model = tf.keras.models.load_model(model_fn)

In [65]:
# pardon my words
sent = ['cina lu', 
        'keren anjing lu', 
        'anjing lu', 
        'gausah banyak bacot', 
        'hahahaha kafir gila', 
        'Jadi cowo itu harus Gantle kalo ga Gantle itu namanya BANCI',
        'Islam Nusantara produk JIL, dipasarkan dg gencar oleh antek JIL. Sementara JIL anti Arab, tapi pro Aseng! Kata (alm) KH. Hasyim Muzadi',
        'kalimat apa gitu',
        'dijaga ya bahasanya']

In [63]:
preprocessed = preprocess(sent)
# preprocessed

In [64]:
pred = model.predict(preprocessed)
np.round(pred, 0)

array([[1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.]], dtype=float32)